In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

pd.options.display.float_format = '{:20,.5f}'.format  ### Setting pd to have a numerical precision up to 5 decimal points

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

## Default SE on the original OLS is HC3, default CI on bootstrapped parameter is BC. 

That is by default se_type = 'hc3', ci_type = 'bc'.

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000, se_type='hc3', ci_type = 'bc')  # se_type = 'hc3' and ci_type = 'bc' are default options for these arguments.
psb.fit()

In [5]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000)
rsb.fit()

In [6]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "standard_normal")
wb_stdn.fit()

In [7]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "rademacher")
wb_rad.fit()

In [8]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen")
wb_mam.fit()

In [9]:
psb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                     Pairs Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                      |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2030   |      6.2012      | 0.0018 |     0.0451    |    0.0436    |     3.39     | [6.1174, 6.2903] |  0.1729 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1739   |      0.1738      | 0.0000 |     0.0050 

In [10]:
rsb.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                    Residual Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                    |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2030   |      6.2038      | 0.0007 |     0.0451    |    0.0354    |    21.49     | [6.1327, 6.2674] |  0.1347 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1739   |      0.1741      | 0.0002 |     0.0050 

In [11]:
wb_stdn.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|           Wild Bootstrap with Standard Normal results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI            |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2030   |      6.2032      | 0.0002 |     0.0451    |    0.0343    |    23.97     | [6.1339, 6.2685] |  0.1345 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1739   |      0.1738      | 0.0000 |     0.0050 

In [12]:
wb_rad.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|              Wild Bootstrap with Rademacher results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI              |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2030   |      6.2023      | 0.0007 |     0.0451    |    0.0368    |    18.36     | [6.1350, 6.2766] |  0.1416 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1739   |      0.1738      | 0.0001 |     0.0050 

In [13]:
wb_mam.summary()

+----------------------------------------------------------------------------------------------------------------------------------+
|                Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI                |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|     Var      | OLS Params | Avg of BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        | CI Diff |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
|    const     |   6.2030   |      6.2034      | 0.0004 |     0.0451    |    0.0354    |    21.50     | [6.1338, 6.2728] |  0.1390 |
+--------------+------------+------------------+--------+---------------+--------------+--------------+------------------+---------+
| tanass_clean |   0.1739   |      0.1738      | 0.0001 |     0.0050 

## Some useful methods were implemented (let's use it on wb_mam object of Wild Bootstrap with Mammen)

- The common in the following three methods is that either a string (if one wishes to capture one variable or ci) or 1-D like array can be provided to 'which_var' and/or 'which_ci' (the latter if exists in that object class).

### get_ci() method

#### Vanila version is when only the actual CI was used with all independent variables

In [14]:
wb_mam.get_ci()

bc                     
                              lwb                  upb
const                     6.13380              6.27276
tanass_clean              0.16653              0.18127
tax                       0.51007              0.53500

#### However, any combination of ('bc', 'bca', 'percentile) CI types could be selected and any combinations of independent variables can be chosen with 'which_ci' and 'which_var' optional arguments, respectively.

Please note that if choosing 'bca' the calculation can take a while as it uses jacknife resampling for calculating the acceleration factor

In [15]:
wb_mam.get_ci(which_ci=['bc', 'percentile'], which_var=['tax', 'const'])

bc                                percentile  \
                       lwb                  upb                  lwb   
tax                0.51007              0.53500              0.51025   
const              6.13380              6.27276              6.13366   

                            
                       upb  
tax                0.53517  
const              6.27276

In [16]:
wb_mam.get_ci(which_ci=['bca', 'bc'], which_var='tanass_clean')

bc                                       bca  \
                              lwb                  upb                  lwb   
tanass_clean              0.16653              0.18127              0.16656   

                                   
                              upb  
tanass_clean              0.18140

In [17]:
wb_mam.get_ci(which_ci='bc', which_var='tanass_clean')

bc                     
                              lwb                  upb
tanass_clean              0.16653              0.18127

### get_all_se() method

The following SE-s are calculated: 
- bootstrapped - standard error of the bootstrapped parameters
- constant - model-based OLS Standard Errors, that is, constant variance is assumed 
- HC0, HC1, HC2, HC3, HC4, HC4m, HC5 - Heteroskedasticity-Consistent Standard Errors (HCE) using sandwich estimators 

#### Vanila version is when using all indepencdent variables. 

In [18]:
wb_mam.get_all_se()

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
const,0.03542,0.03534,0.04510,0.04511,0.04512,0.04513,0.04513,0.04513,0.04513
tanass_clean,0.00390,0.00390,0.00500,0.00500,0.00500,0.00500,0.00501,0.00501,0.00501
tax,0.00652,0.00654,0.00902,0.00902,0.00902,0.00902,0.00903,0.00903,0.00903


#### A subset of indepent variables can be chosen with 'which_var' argument to calculate the above-mentioned 9 SE-s

In [19]:
wb_mam.get_all_se(which_var=['tanass_clean', 'tax'])

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tanass_clean,0.00390,0.00390,0.00500,0.00500,0.00500,0.00500,0.00501,0.00501,0.00501
tax,0.00652,0.00654,0.00902,0.00902,0.00902,0.00902,0.00903,0.00903,0.00903


In [20]:
wb_mam.get_all_se(which_var='tax')

,bootstrapped,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5
tax,0.00652,0.00654,0.00902,0.00902,0.00902,0.00902,0.00903,0.00903,0.00903


### get_bootstrap_params() method

#### Vanila version: Returning a dataframe capturing the parameter estimate of ALL each independent variables in each (wild) bootstrap.

In [21]:
wb_mam.get_bootstrap_params()

,const,tanass_clean,tax
0,6.17783,0.17751,0.51911
1,6.13784,0.18012,0.52724
2,6.16870,0.17626,0.52511
3,6.26846,0.17586,0.50987
4,6.17768,0.17875,0.52023
...,...,...,...
995,6.08984,0.18517,0.52929
996,6.22827,0.17306,0.51343
997,6.21688,0.17489,0.51683
998,6.20410,0.17807,0.51713


#### As usual, the desired independent variable can be chosen with 'which_var' argument

In [22]:
wb_mam.get_bootstrap_params(which_var='tax')

,tax
0,0.51911
1,0.52724
2,0.52511
3,0.50987
4,0.52023
...,...
995,0.52929
996,0.51343
997,0.51683
998,0.51713


In [23]:
wb_mam.get_bootstrap_params(which_var=('const', 'tanass_clean'))

,const,tanass_clean
0,6.17783,0.17751
1,6.13784,0.18012
2,6.16870,0.17626
3,6.26846,0.17586
4,6.17768,0.17875
...,...,...
995,6.08984,0.18517
996,6.22827,0.17306
997,6.21688,0.17489
998,6.20410,0.17807


### bp_test() and white_test() methods

#### Vanila version: for bp_test() returning the robust version (proposed by Roger Koenker) of this test when the presence of the heteroscedasticity is examined on the residual of the OLS on the original sample data. For white_test(), there are no optional arguments

In [24]:
wb_mam.bp_test()

(471.4999600925629,
 4.121777622602409e-103,
 247.341401128395,
 1.4261454830003312e-105)

In [25]:
wb_mam.white_test()

(1577.3564210084555, 0.0, 374.3266570338708, 0.0)

#### The non-robust version of bp_test() can be achieved by setting bp_test(robust = False).

In [26]:
wb_mam.bp_test(robust = False)

(989.0384882393482,
 1.7100983338654616e-215,
 247.34140112839475,
 1.4261454830003312e-105)

At each test, the first value is the calculated LM-test stats, the second is the 'LM p-value' corresponding to the LM-test stats, the third value is the F-test stats and last one is the 'F p-value'.

As it can be seen the p-values are 0, so we can rejcet the null hypothesis of homoscedasticity, and accept the alternative hypothesis that the variance of the residuals is heteroscedastic.